#Eligibility

In [108]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [109]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


*Subject, admission 등 입원 정보 테이블에서 대상환자를 추출하겠습니다.

In [110]:
curs.execute("select * from mimiciv_hosp.admissions;")
columns_name = [desc[0] for desc in curs.description]
admissions = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [111]:
curs.execute("select * from mimiciv_hosp.patients;")
columns_name = [desc[0] for desc in curs.description]
patients = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [112]:
curs.execute("select * from mimiciv_hosp.transfers;")
columns_name = [desc[0] for desc in curs.description]
transfers = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [113]:
curs.execute("select * from mimiciv_icu.icustays;")
columns_name = [desc[0] for desc in curs.description]
icustays = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [114]:
admissions.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0


In [115]:
patients.head(2)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None


In [116]:
transfers.head(2)

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27


In [117]:
icustays.head(2)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535


성인환자를 조회해봅시다

In [118]:
adults_pat = patients >> filter_by(X.anchor_age >= 18)
adults_pat
#299712

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000068,F,19,2160,2008 - 2010,None
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,None
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,None
299708,19999829,F,28,2186,2008 - 2010,None
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,None


In [119]:
#60세 이상 노인은 추후 subgroup 으로 분석될 예정임, 확인만
elder_pat = adults_pat >> filter_by(X.anchor_age >= 60)
elder_pat
# 98065

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
3,10000084,M,72,2160,2017 - 2019,2161-02-13
12,10000459,M,63,2178,2011 - 2013,None
13,10000473,M,81,2138,2017 - 2019,None
19,10000635,F,74,2136,2014 - 2016,None
23,10000764,M,86,2132,2014 - 2016,None
...,...,...,...,...,...,...
299689,19999287,F,71,2191,2008 - 2010,2197-09-02
299692,19999303,F,61,2161,2008 - 2010,None
299695,19999379,F,91,2174,2011 - 2013,None
299699,19999565,F,66,2185,2008 - 2010,None


In [120]:
adults_hadm = adults_pat >> left_join(admissions, by = "subject_id") >> select("subject_id", "gender", "anchor_age", "hadm_id", "admittime", "dischtime", "deathtime")
adults_hadm
#성인 입원정보를 불러옵니다. #550210 왜 늘어나냐면 1명이 여러번 입원했기 때문임

,subject_id,gender,anchor_age,hadm_id,admittime,dischtime,deathtime
0,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT
1,10000032,F,52,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT
2,10000032,F,52,25742920.0,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT
3,10000032,F,52,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT
4,10000048,F,23,NaN,NaT,NaT,NaT
...,...,...,...,...,...,...,...
550205,19999829,F,28,NaN,NaT,NaT,NaT
550206,19999840,M,58,21033226.0,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00
550207,19999840,M,58,26071774.0,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT
550208,19999914,F,49,NaN,NaT,NaT,NaT


In [121]:
adults_hadm.dtypes #데이터타입확인

subject_id             int64
gender                object
anchor_age             int64
hadm_id              float64
admittime     datetime64[ns]
dischtime     datetime64[ns]
deathtime     datetime64[ns]
dtype: object

In [122]:
transfers.dtypes
#df=df.astype({"colName":"type"})
#nicu1 = newborn_a1[newborn_a1['careunit'].str.contains('tensive', na = False)]
#nicu = nicu1 >> drop(X.subject_id_y) >> rename(subject_id = X.subject_id_x)

subject_id              int64
hadm_id               float64
transfer_id             int64
eventtype              object
careunit               object
intime         datetime64[ns]
outtime        datetime64[ns]
dtype: object

In [123]:
adult_transfers = adults_hadm >> left_join(transfers, by = ("subject_id", "hadm_id"))
adult_transfers 
#transfer에는 병동이동정보가 담겨 있습니다. 붙여보면 icu 등 이동 이력이 붙어서 더 늘어난 것을 확인할 수 있습니다.

,subject_id,gender,anchor_age,hadm_id,admittime,dischtime,deathtime,transfer_id,eventtype,careunit,intime,outtime
0,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,36904543,discharge,None,2180-05-07 17:21:27,NaT
3,10000032,F,52,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,34100253,discharge,None,2180-06-27 18:49:12,NaT
4,10000032,F,52,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
...,...,...,...,...,...,...,...,...,...,...,...,...
1723451,19999914,F,49,NaN,NaT,NaT,NaT,32002659,ED,Emergency Department,2158-12-24 11:41:00,2158-12-24 11:56:00
1723452,19999987,F,57,23865745.0,2145-11-02 21:38:00,2145-11-11 12:57:00,NaT,30249304,transfer,Neurology,2145-11-04 21:29:30,2145-11-11 13:00:47
1723453,19999987,F,57,23865745.0,2145-11-02 21:38:00,2145-11-11 12:57:00,NaT,34731548,ED,Emergency Department,2145-11-02 19:28:00,2145-11-02 22:59:00
1723454,19999987,F,57,23865745.0,2145-11-02 21:38:00,2145-11-11 12:57:00,NaT,36195440,admit,Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30


나이로 환자 뽑고 => 조건에 따라 제거해 나가기 -> 필요한 환자만 남기기

In [124]:
#나이로 환자 뽑고 -> 조건에따라 제거해 나가기 -> 필요한 환자만 남기기
# 아까 어덜트로 다시가볼까요

In [125]:
#adults_hadm.info
#adults_hadm.isnull()
#adults_hadm 에서 입원을 하지 않은 환자는 필요없으므로, hadm_id가 없는 환자들은 지워주도록 합시다 
adults_hadm2 = adults_hadm >> mask(adults_hadm.hadm_id.notnull())
adults_hadm2

,subject_id,gender,anchor_age,hadm_id,admittime,dischtime,deathtime
0,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT
1,10000032,F,52,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT
2,10000032,F,52,25742920.0,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT
3,10000032,F,52,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT
5,10000068,F,19,25022803.0,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT
...,...,...,...,...,...,...,...
550203,19999828,F,46,25744818.0,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT
550204,19999828,F,46,29734428.0,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT
550206,19999840,M,58,21033226.0,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00
550207,19999840,M,58,26071774.0,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT


In [126]:
#그리고나는 icu에 입원했던 정보가 필요하니까, icu 데이터를 붙여봅니다
adults_icu = adults_hadm2 >> left_join(icustays, by = ("subject_id", "hadm_id"))
adults_icu

,subject_id,gender,anchor_age,hadm_id,admittime,dischtime,deathtime,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,F,52,22595853.0,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,10000032,F,52,22841357.0,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,10000032,F,52,25742920.0,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,10000032,F,52,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,39553978.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
4,10000068,F,19,25022803.0,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438168,19999828,F,46,25744818.0,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,36075953.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
438169,19999828,F,46,29734428.0,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN
438170,19999840,M,58,21033226.0,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,38978960.0,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766
438171,19999840,M,58,26071774.0,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [127]:
#stayid 가 없는 사람이 많으니까 없애야겠죠?
adults_icu2 = adults_icu >> mask(adults_icu.stay_id.notnull())
adults_icu2
#이쯤되니 index 가 거슬리기 시작합니다... 이걸 교정할지는 좀 생각해봅시다.

,subject_id,gender,anchor_age,hadm_id,admittime,dischtime,deathtime,stay_id,first_careunit,last_careunit,intime,outtime,los
3,10000032,F,52,29079034.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,39553978.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
32,10000980,F,73,26913865.0,2189-06-27 07:38:00,2189-07-03 03:00:00,NaT,39765666.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
39,10001217,F,55,24597018.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,37067082.0,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
40,10001217,F,55,27703517.0,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,34592300.0,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
58,10001725,F,46,25563031.0,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,31205490.0,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438142,19999442,M,41,26785317.0,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,32336619.0,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
438147,19999625,M,81,25304202.0,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,31070865.0,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
438168,19999828,F,46,25744818.0,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,36075953.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
438170,19999840,M,58,21033226.0,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,38978960.0,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [128]:
#일단 여기까지 한것을 저장해주겠습니다
adults_icu2.to_csv('../outputs/adult_icu2.csv', index=False)

In [129]:
#어떤 스키마에 어떤 테이블이 있는지 확인
curs.execute("select * from PG_TABLES;")
rows = curs.fetchall( ) #결과가지고옴

for row in rows:
    print(row)

('mimiciv_hosp', 'diagnoses_icd', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'labevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_icd_diagnoses', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'd_icd_procedures', 'mai_admin', None, True, False, False, False)
('mimiciv_hosp', 'emar', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'hcpcsevents', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'd_labitems', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'pharmacy', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'omr', 'mai_admin', None, False, False, False, False)
('mimiciv_hosp', 'poe_detail', 'mai_admin', None, True, False, True, False)
('mimiciv_hosp', 'poe', 'mai_admin', None, True, False, True, False)
('pg_catalog', 'pg_statistic', 'postgres', None, True, False, False, False)
('pg_catalog', 'pg_type', 'postgres', None, True, False, False, False)
('mimiciv_hosp', 'pro

extubation 을 할 수 있는 사람은 기관 삽관 이력이 있어야 합니다.그럼 intubation, extubation 이력을 찾아야 합니다. 이 정보는 시술 정보에 있을 것입니다.
d_로 시작하는 테이블들은 dictionary 정보들을 담고 있습니다. 아마 시술정보는 item이나 procedure에 있을 것입니다.

In [130]:
# curs.execute("select * from mimiciv_hosp.d_icd_diagnoses;")
# columns_name = [desc[0] for desc in curs.description]
# d_icd_diagnoses = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# d_icd_diagnoses

In [131]:
# curs.execute("select * from mimiciv_hosp.diagnoses_icd;")
# columns_name = [desc[0] for desc in curs.description]
# diagnoses_icd = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# diagnoses_icd

In [132]:
# curs.execute("select * from mimiciv_hosp.d_labitems;")
# columns_name = [desc[0] for desc in curs.description]
# d_labitems = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# d_labitems

In [133]:
curs.execute("select * from mimiciv_hosp.d_icd_procedures;")
columns_name = [desc[0] for desc in curs.description]
d_icd_procedures = pd.DataFrame(curs.fetchall(), columns=columns_name) 
d_icd_procedures

,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"
...,...,...,...
85252,XW0DXV5,10,Introduction of Gilteritinib Antineoplastic in...
85253,XXE,10,"New Technology, Physiological Systems, Measure..."
85254,XXE5XM5,10,"Measurement of Infection, Whole Blood Nucleic ..."
85255,XY0,10,"New Technology, Extracorporeal, Introduction"


In [134]:
curs.execute("select * from mimiciv_hosp.procedures_icd;")
columns_name = [desc[0] for desc in curs.description]
procedures_icd = pd.DataFrame(curs.fetchall(), columns=columns_name) 
procedures_icd

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10
...,...,...,...,...,...,...
669181,19999840,21033226,5,2164-09-16,0331,9
669182,19999840,26071774,1,2164-07-25,8891,9
669183,19999840,26071774,2,2164-07-25,8841,9
669184,19999987,23865745,1,2145-11-07,8841,9


In [135]:
curs.execute("select * from mimiciv_icu.d_items;")
columns_name = [desc[0] for desc in curs.description]
d_items = pd.DataFrame(curs.fetchall(), columns=columns_name) 
d_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,None,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,None,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4009,230077,Documented on DC Summary (PICC),Documented on DC Summary (PICC),chartevents,PICC Line Insertion,None,Text,NaN,NaN
4010,230078,Documented on DC Summary (Thora),Documented on DC Summary (Thora),chartevents,Thoracentesis,None,Text,NaN,NaN
4011,230083,Replacement Fluid (Pre-Filter),Replacement Fluid (Pre-Filter),chartevents,Dialysis,None,Text,NaN,NaN
4012,230084,Replacement Fluid (Post-Filter),Replacement Fluid (Post-Filter),chartevents,Dialysis,None,Text,NaN,NaN


In [136]:
# engine = create_engine('postgresql+psycopg2://mai_admin:admin1q2w!@@local/mimic4')
# #진단/시술관련 정보 가지고와서 환자 정의에 활용하기 - 변수 찾아보기
# d_icd_diagnoses = pd.read_sql("select * from mimic4.mimic_hosp.d_icd_diagnoses", engine)
# d_icd_procedures = pd.read_sql("select * from mimic4.mimic_hosp.d_icd_procedures", engine)
# diagnoses_icd = pd.read_sql("select * from mimic4.mimic_hosp.diagnoses_icd", engine)
# procedures_icd = pd.read_sql("select * from mimic4.mimic_hosp.procedures_icd", engine)
# d_labitems = pd.read_sql("select * from mimic4.mimic_hosp.d_labitems", engine)
# d_items = pd.read_sql("select * from mimic4.mimic_icu.d_items", engine) 

In [137]:
d_icd_procedures.head(2)

,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart


In [138]:
procedures_icd.head(2)

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9


In [139]:
d_icd_procedures[d_icd_procedures['long_title'].str.contains('ubation')]


,icd_code,icd_version,long_title
25019,0944,9,Intubation of nasolacrimal duct
77678,9605,9,Other intubation of respiratory tract


In [140]:
d_icd_procedures[d_icd_procedures['long_title'].str.contains('Exubation')]

,icd_code,icd_version,long_title


In [141]:
d_items[d_items['label'].str.contains('Intubation')] #intubation, extubation 은 chartevents 에 있는 것으로 봐야 할 듯 함. // Procedureevents 에도 있음 

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
321,222871,Mallampati classification (Intubation),Mallampati classification (Intubation),chartevents,Intubation,None,Text,NaN,NaN
322,223059,Intubation - Details,Intubation - Details,chartevents,Respiratory,None,Text,NaN,NaN
665,224385,Intubation,Intubation,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1185,225271,Induction/Drugs/Doses (Intubation),Induction/Drugs/Doses (Intubation),chartevents,Intubation,None,Text,NaN,NaN
1198,225291,Indication (Intubation),Indication (Intubation),chartevents,Intubation,None,Text,NaN,NaN
1199,225292,Unable to complete Airway Assessment (Intubation),Unable to complete Airway (Intubation),chartevents,Intubation,None,Checkbox,NaN,NaN
1200,225294,Mouth Opening (Intubation),Mouth Opening (Intubation),chartevents,Intubation,None,Text,NaN,NaN
1201,225295,Thyromental Distance (Intubation),Thyromental Distance (Intubation),chartevents,Intubation,None,Text,NaN,NaN
1202,225296,Mandibular Prognatism (Intubation),Mandibular Prognatism (Intubation),chartevents,Intubation,None,Text,NaN,NaN
1203,225297,Teeth/Dentition (Intubation),Teeth/Dentition (Intubation),chartevents,Intubation,None,Text,NaN,NaN


In [142]:
d_items[d_items['label'].str.contains('Extubation')] #intubation, extubation 은 chartevents 에 있는 것으로 봐야 할 듯 함. // Procedureevents 에도 있음 

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
1324,225468,Unplanned Extubation (patient-initiated),Unplanned Extubation (patient-initiated),procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1331,225477,Unplanned Extubation (non-patient initiated),Unplanned Extubation (non-patient initiated),procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
2141,227194,Extubation,Extubation,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN


* Intubation 은
224385	Intubation	Intubation	procedureevents	1-Intubation/Extubation (223059	Intubation - Details	Intubation - Details	chartevents	Respiratory	는 참고 / 225301	RSI (Intubation)	RSI (Intubation)	chartevents	Intubation	None	Checkbox	참고)
* Extubation 은
227194	Extubation	Extubation	procedureevents	1-Intubation/Extubation	(225477	Unplanned Extubation (non-patient initiated)	Unplanned Extubation (non-patient initiated)	,225468	Unplanned Extubation (patient-initiated)	Unplanned Extubation (patient-initiated) )

In [143]:
curs.execute("select * from mimiciv_icu.procedureevents where itemid in (224385);")
columns_name = [desc[0] for desc in curs.description]
intubation_all = pd.DataFrame(curs.fetchall(), columns=columns_name) 
intubation_all

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,42150,2131-01-11 04:30:00,2131-01-11 04:31:00,2131-01-11 05:40:00,224385,1.0,None,...,2338046,2338046,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10003400,23559586,38383343,36212,2137-08-17 21:21:00,2137-08-17 21:22:00,2137-08-17 21:21:00,224385,1.0,None,...,9156137,9156137,Intubation/Extubation,Task,90.5,0,0,FinishedRunning,1.0,0.0
2,10004401,27939719,31202136,58760,2144-04-11 05:03:00,2144-04-11 05:04:00,2144-04-11 05:03:00,224385,1.0,None,...,991564,991564,Intubation/Extubation,Task,120.0,0,0,FinishedRunning,1.0,0.0
3,10004401,29988601,32773003,38908,2144-01-27 19:00:00,2144-01-27 19:01:00,2144-01-27 19:12:00,224385,1.0,None,...,4299790,4299790,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
4,10004401,29988601,32773003,42665,2144-01-30 13:00:00,2144-01-30 13:01:00,2144-01-30 13:05:00,224385,1.0,None,...,6177150,6177150,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8483,19989918,20256395,35582926,50512,2175-09-23 14:15:00,2175-09-23 14:16:00,2175-09-23 14:27:00,224385,1.0,None,...,7877602,7877602,Intubation/Extubation,Task,124.4,0,0,FinishedRunning,1.0,0.0
8484,19997293,20208898,38350444,58760,2123-10-20 04:05:00,2123-10-20 04:06:00,2123-10-20 04:39:00,224385,1.0,None,...,4889483,4889483,Intubation/Extubation,Task,134.5,0,0,FinishedRunning,1.0,0.0
8485,19997367,20617667,35616526,10976,2126-04-26 20:14:00,2126-04-26 20:15:00,2126-04-26 21:14:00,224385,1.0,None,...,5095062,5095062,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0
8486,19997367,20617667,35616526,27479,2126-05-05 18:20:00,2126-05-05 18:21:00,2126-05-05 18:20:00,224385,1.0,None,...,8462542,8462542,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0


In [144]:
curs.execute("select * from mimiciv_icu.procedureevents where itemid in (227194,225477,225468);")
columns_name = [desc[0] for desc in curs.description]
extubation_all = pd.DataFrame(curs.fetchall(), columns=columns_name) 
extubation_all
#아 개수가 안맞네....

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,35966,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,None,...,6470885,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10002013,23581541,39060235,65197,2160-05-18 18:00:00,2160-05-18 18:01:00,2160-05-18 17:59:00,227194,1.0,None,...,3546453,3546453,Intubation/Extubation,Task,96.0,0,0,FinishedRunning,1.0,0.0
2,10002428,23473524,35479615,42665,2156-05-20 10:45:00,2156-05-20 10:46:00,2156-05-20 10:51:00,227194,1.0,None,...,9794232,9794232,Intubation/Extubation,Task,48.4,0,0,FinishedRunning,1.0,0.0
3,10002428,28662225,38875437,3009,2156-04-22 17:10:00,2156-04-22 17:11:00,2156-04-22 17:10:00,227194,1.0,None,...,9954431,9954431,Intubation/Extubation,Task,43.0,0,0,FinishedRunning,1.0,0.0
4,10002760,28094813,31831386,50512,2141-04-21 08:00:00,2141-04-21 08:01:00,2141-04-21 08:04:00,227194,1.0,None,...,2857656,2857656,Intubation/Extubation,Task,91.0,0,0,FinishedRunning,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23117,19998330,21135114,31417783,62592,2178-10-22 14:54:00,2178-10-22 14:55:00,2178-10-22 14:54:00,227194,1.0,None,...,4056693,4056693,Intubation/Extubation,Task,77.6,0,0,FinishedRunning,1.0,0.0
23118,19998330,24492004,32641669,10134,2178-10-02 08:32:00,2178-10-02 08:33:00,2178-10-02 08:32:00,225468,1.0,None,...,5460688,5460688,Significant Events,Task,77.1,0,0,FinishedRunning,1.0,0.0
23119,19999068,21606769,30143796,99850,2161-08-28 13:35:00,2161-08-28 13:36:00,2161-08-28 13:37:00,227194,1.0,None,...,489079,489079,Intubation/Extubation,Task,55.8,0,0,FinishedRunning,1.0,0.0
23120,19999287,20175828,35165301,50512,2197-08-07 09:20:00,2197-08-07 09:21:00,2197-08-07 09:23:00,227194,1.0,None,...,4142066,4142066,Intubation/Extubation,Task,86.2,0,0,FinishedRunning,1.0,0.0


In [145]:
d_items[d_items['category'].str.contains('1-Intubation/Extubation')]  #재확인

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
665,224385,Intubation,Intubation,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1308,225448,Percutaneous Tracheostomy,Percutaneous Tracheostomy,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1324,225468,Unplanned Extubation (patient-initiated),Unplanned Extubation (patient-initiated),procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1331,225477,Unplanned Extubation (non-patient initiated),Unplanned Extubation (non-patient initiated),procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
1808,226237,Open Tracheostomy,Open Tracheostomy,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN
2141,227194,Extubation,Extubation,procedureevents,1-Intubation/Extubation,None,Processes,NaN,NaN


In [146]:
intubation_all.to_csv('../outputs/intubation_all.csv', index=False)
intubation_all = pd.read_csv('../outputs/intubation_all.csv')
intubation_all

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,42150,2131-01-11 04:30:00,2131-01-11 04:31:00,2131-01-11 05:40:00,224385,1.0,NaN,...,2338046,2338046,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10003400,23559586,38383343,36212,2137-08-17 21:21:00,2137-08-17 21:22:00,2137-08-17 21:21:00,224385,1.0,NaN,...,9156137,9156137,Intubation/Extubation,Task,90.5,0,0,FinishedRunning,1.0,0.0
2,10004401,27939719,31202136,58760,2144-04-11 05:03:00,2144-04-11 05:04:00,2144-04-11 05:03:00,224385,1.0,NaN,...,991564,991564,Intubation/Extubation,Task,120.0,0,0,FinishedRunning,1.0,0.0
3,10004401,29988601,32773003,38908,2144-01-27 19:00:00,2144-01-27 19:01:00,2144-01-27 19:12:00,224385,1.0,NaN,...,4299790,4299790,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
4,10004401,29988601,32773003,42665,2144-01-30 13:00:00,2144-01-30 13:01:00,2144-01-30 13:05:00,224385,1.0,NaN,...,6177150,6177150,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8483,19989918,20256395,35582926,50512,2175-09-23 14:15:00,2175-09-23 14:16:00,2175-09-23 14:27:00,224385,1.0,NaN,...,7877602,7877602,Intubation/Extubation,Task,124.4,0,0,FinishedRunning,1.0,0.0
8484,19997293,20208898,38350444,58760,2123-10-20 04:05:00,2123-10-20 04:06:00,2123-10-20 04:39:00,224385,1.0,NaN,...,4889483,4889483,Intubation/Extubation,Task,134.5,0,0,FinishedRunning,1.0,0.0
8485,19997367,20617667,35616526,10976,2126-04-26 20:14:00,2126-04-26 20:15:00,2126-04-26 21:14:00,224385,1.0,NaN,...,5095062,5095062,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0
8486,19997367,20617667,35616526,27479,2126-05-05 18:20:00,2126-05-05 18:21:00,2126-05-05 18:20:00,224385,1.0,NaN,...,8462542,8462542,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0


In [147]:
extubation_all.to_csv('../outputs/extubation_all.csv', index=False)
extubation_all = pd.read_csv('../outputs/extubation_all.csv')
extubation_all

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,35966,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,NaN,...,6470885,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10002013,23581541,39060235,65197,2160-05-18 18:00:00,2160-05-18 18:01:00,2160-05-18 17:59:00,227194,1.0,NaN,...,3546453,3546453,Intubation/Extubation,Task,96.0,0,0,FinishedRunning,1.0,0.0
2,10002428,23473524,35479615,42665,2156-05-20 10:45:00,2156-05-20 10:46:00,2156-05-20 10:51:00,227194,1.0,NaN,...,9794232,9794232,Intubation/Extubation,Task,48.4,0,0,FinishedRunning,1.0,0.0
3,10002428,28662225,38875437,3009,2156-04-22 17:10:00,2156-04-22 17:11:00,2156-04-22 17:10:00,227194,1.0,NaN,...,9954431,9954431,Intubation/Extubation,Task,43.0,0,0,FinishedRunning,1.0,0.0
4,10002760,28094813,31831386,50512,2141-04-21 08:00:00,2141-04-21 08:01:00,2141-04-21 08:04:00,227194,1.0,NaN,...,2857656,2857656,Intubation/Extubation,Task,91.0,0,0,FinishedRunning,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23117,19998330,21135114,31417783,62592,2178-10-22 14:54:00,2178-10-22 14:55:00,2178-10-22 14:54:00,227194,1.0,NaN,...,4056693,4056693,Intubation/Extubation,Task,77.6,0,0,FinishedRunning,1.0,0.0
23118,19998330,24492004,32641669,10134,2178-10-02 08:32:00,2178-10-02 08:33:00,2178-10-02 08:32:00,225468,1.0,NaN,...,5460688,5460688,Significant Events,Task,77.1,0,0,FinishedRunning,1.0,0.0
23119,19999068,21606769,30143796,99850,2161-08-28 13:35:00,2161-08-28 13:36:00,2161-08-28 13:37:00,227194,1.0,NaN,...,489079,489079,Intubation/Extubation,Task,55.8,0,0,FinishedRunning,1.0,0.0
23120,19999287,20175828,35165301,50512,2197-08-07 09:20:00,2197-08-07 09:21:00,2197-08-07 09:23:00,227194,1.0,NaN,...,4142066,4142066,Intubation/Extubation,Task,86.2,0,0,FinishedRunning,1.0,0.0


In [148]:
extubation_all['value'].unique()

array([1.])

In [149]:
extubation_all['value'].value_counts()

value
1.0    23122
Name: count, dtype: int64

In [150]:
#이제 이 데이터를 활용해서 환자를 정의해보겠습니다.
#먼저 intubation -> extubation -> intubation(reintubation) 순으로 절차가 이루어 집니다. 여기서 intubation~extubation 이 적어도 24시간 이상 이루어진 환자를 찾습니다 (일시적 목적으로 사용한 환자 제거)
#그다음, extubation 이후 48시간 이내에 다시 intubation 을 한 환자를 찾습니다. (reintubation) 이 환자들은 extubation failure 군으로 분류 될 것입니다.
#그렇다면 extubation 이후 48시간을 초과하여 재 삽관(reintubation 한 환자는 extubation non-failure 로 분류 할 것입니다. sucess라고 단정지을 수 있는)

In [151]:
intubation_all

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,42150,2131-01-11 04:30:00,2131-01-11 04:31:00,2131-01-11 05:40:00,224385,1.0,NaN,...,2338046,2338046,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10003400,23559586,38383343,36212,2137-08-17 21:21:00,2137-08-17 21:22:00,2137-08-17 21:21:00,224385,1.0,NaN,...,9156137,9156137,Intubation/Extubation,Task,90.5,0,0,FinishedRunning,1.0,0.0
2,10004401,27939719,31202136,58760,2144-04-11 05:03:00,2144-04-11 05:04:00,2144-04-11 05:03:00,224385,1.0,NaN,...,991564,991564,Intubation/Extubation,Task,120.0,0,0,FinishedRunning,1.0,0.0
3,10004401,29988601,32773003,38908,2144-01-27 19:00:00,2144-01-27 19:01:00,2144-01-27 19:12:00,224385,1.0,NaN,...,4299790,4299790,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
4,10004401,29988601,32773003,42665,2144-01-30 13:00:00,2144-01-30 13:01:00,2144-01-30 13:05:00,224385,1.0,NaN,...,6177150,6177150,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8483,19989918,20256395,35582926,50512,2175-09-23 14:15:00,2175-09-23 14:16:00,2175-09-23 14:27:00,224385,1.0,NaN,...,7877602,7877602,Intubation/Extubation,Task,124.4,0,0,FinishedRunning,1.0,0.0
8484,19997293,20208898,38350444,58760,2123-10-20 04:05:00,2123-10-20 04:06:00,2123-10-20 04:39:00,224385,1.0,NaN,...,4889483,4889483,Intubation/Extubation,Task,134.5,0,0,FinishedRunning,1.0,0.0
8485,19997367,20617667,35616526,10976,2126-04-26 20:14:00,2126-04-26 20:15:00,2126-04-26 21:14:00,224385,1.0,NaN,...,5095062,5095062,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0
8486,19997367,20617667,35616526,27479,2126-05-05 18:20:00,2126-05-05 18:21:00,2126-05-05 18:20:00,224385,1.0,NaN,...,8462542,8462542,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0


In [152]:
intubation_all['statusdescription'].value_counts()

statusdescription
FinishedRunning    8488
Name: count, dtype: int64

In [153]:
intubation1 =intubation_all >> select("subject_id", "hadm_id", "stay_id", "starttime", "itemid", "patientweight")

In [154]:
extubation_all['statusdescription'].value_counts()


statusdescription
FinishedRunning    23122
Name: count, dtype: int64

In [155]:
extubation1 = extubation_all >> select("subject_id", "hadm_id", "stay_id", "starttime", "itemid", "patientweight")

In [156]:
#extubation1.rename(columns={'itemid':'extubationcuase'}, inplace=True)

In [157]:
#extubation
#1227194 Extubation	Extubation	procedureevents	1-Intubation/Extubation
#225477	Unplanned Extubation (non-patient initiated)
#225468	Unplanned Extubation (patient-initiated)

def label_extubation(row):
    if row['itemid'] == 225477:
        return 'Unplanned Extuabtion (non-patient initiated)'
    elif row['itemid'] == 225468:
        return 'Unplanned Extuabtion (patient-initiated)'
    else:
        return 'Planned Extubation'
    
extubation1['extubationcause'] = extubation1.apply(lambda row: label_extubation(row), axis=1)

In [158]:
extubation1['extubationcause'].value_counts()

extubationcause
Planned Extubation                              22336
Unplanned Extuabtion (patient-initiated)          730
Unplanned Extuabtion (non-patient initiated)       56
Name: count, dtype: int64

In [159]:
#두개의 테이블이 되었으니 먼저 ventilator 를 24시간 이상 유지한 환자만 골라낼 것임
#먼저 starttime이라는 컬럼명이 겹치니까 바꿔 줄 예정
intubation1.rename(columns={'starttime':'intubationtime'}, inplace=True)
extubation1.rename(columns={'starttime':'extubationtime'}, inplace=True)

In [160]:
#extubation time에서 intubation time을 빼면, 양수라면 intubation~extubation까지의 시간이 나옴, intubation time 이 없으면, duration을 알 수 없으니 데이터가 빠져야 하고
#-가 나오면 intubation time 이 더 크다는 소리이니 빠지면 됨.

In [161]:
#extubation1테이블에 intubation1을 붙인다
extubation_intubation = extubation1 >> left_join(intubation1, by = ("subject_id", "hadm_id", "stay_id"))

In [163]:
extubation_intubation
extubation_intubation.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25011 entries, 0 to 25010
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       25011 non-null  int64  
 1   hadm_id          25011 non-null  int64  
 2   stay_id          25011 non-null  int64  
 3   extubationtime   25011 non-null  object 
 4   itemid_x         25011 non-null  int64  
 5   patientweight_x  25011 non-null  float64
 6   extubationcause  25011 non-null  object 
 7   intubationtime   8828 non-null   object 
 8   itemid_y         8828 non-null   float64
 9   patientweight_y  8828 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 1.9+ MB


2023-07-01 12:05:15,153| ERROR   | Socket exception: Operation timed out (60)


In [168]:
extubation_intubation.dtypes #extubationtime이랑 intubationtime을 datetime으로 바꿔줘야함

subject_id                              int64
hadm_id                                 int64
stay_id                                 int64
extubationtime                 datetime64[ns]
int_itemid                              int64
int_weight                            float64
extubationcause                        object
intubationtime                 datetime64[ns]
ext_itemid                            float64
ext_weight                            float64
extubationtointubationtime    timedelta64[ns]
dtype: object

In [164]:
#extubationtime이랑 intubationtime을 datetime으로 바꿔줘야함
extubation_intubation['extubationtime'] = pd.to_datetime(extubation_intubation['extubationtime'])
extubation_intubation['intubationtime'] = pd.to_datetime(extubation_intubation['intubationtime'])

In [165]:
#여기서 extubation에서 intubation time을 빼면 몇시간인지 나올텐데 이걸 extubationtointubationtime으로 컬럼을 생성할 것임 근데 그냥하면 에러남
extubation_intubation['extubationtointubationtime'] = extubation_intubation['intubationtime'] - extubation_intubation['extubationtime']

In [167]:
#일단 중복 컬럼명들을 좀 손봐주겠음
extubation_intubation.rename(columns={'itemid_x':'int_itemid', 'itemid_y':'ext_itemid', 'patientweight_x':'int_weight', 'patientweight_y':'ext_weight'}, inplace=True)

In [169]:
extubation_intubation.dtypes

subject_id                              int64
hadm_id                                 int64
stay_id                                 int64
extubationtime                 datetime64[ns]
int_itemid                              int64
int_weight                            float64
extubationcause                        object
intubationtime                 datetime64[ns]
ext_itemid                            float64
ext_weight                            float64
extubationtointubationtime    timedelta64[ns]
dtype: object

In [170]:
extubation_intubation['extubationtointubationtime'] = extubation_intubation['extubationtime'] - extubation_intubation['intubationtime']

In [171]:
extubation_intubation 

,subject_id,hadm_id,stay_id,extubationtime,int_itemid,int_weight,extubationcause,intubationtime,ext_itemid,ext_weight,extubationtointubationtime
0,10001884,26184834,37510196,2131-01-12 17:40:00,227194,65.0,Planned Extubation,2131-01-11 04:30:00,224385.0,65.0,1 days 13:10:00
1,10002013,23581541,39060235,2160-05-18 18:00:00,227194,96.0,Planned Extubation,NaT,NaN,NaN,NaT
2,10002428,23473524,35479615,2156-05-20 10:45:00,227194,48.4,Planned Extubation,NaT,NaN,NaN,NaT
3,10002428,28662225,38875437,2156-04-22 17:10:00,227194,43.0,Planned Extubation,NaT,NaN,NaN,NaT
4,10002760,28094813,31831386,2141-04-21 08:00:00,227194,91.0,Planned Extubation,NaT,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...
25006,19998330,21135114,31417783,2178-10-22 14:54:00,227194,77.6,Planned Extubation,NaT,NaN,NaN,NaT
25007,19998330,24492004,32641669,2178-10-02 08:32:00,225468,77.1,Unplanned Extuabtion (patient-initiated),NaT,NaN,NaN,NaT
25008,19999068,21606769,30143796,2161-08-28 13:35:00,227194,55.8,Planned Extubation,2161-08-25 15:34:00,224385.0,55.8,2 days 22:01:00
25009,19999287,20175828,35165301,2197-08-07 09:20:00,227194,86.2,Planned Extubation,NaT,NaN,NaN,NaT


In [172]:
#그리고 nat나 음수 데이터는 없애야 하는데, 이를 위해 mask를 사용할 것임nat는 intubationtime 이 없으면 지워줄 것이고, intubationtime이 extubationtime보다 작은 경우도 지워줄 것임
extubation_intubation2 = extubation_intubation >> mask(extubation_intubation.intubationtime.notnull(), extubation_intubation.extubationtime.notnull(), extubation_intubation.extubationtointubationtime > timedelta(days=0))
extubation_intubation2


,subject_id,hadm_id,stay_id,extubationtime,int_itemid,int_weight,extubationcause,intubationtime,ext_itemid,ext_weight,extubationtointubationtime
0,10001884,26184834,37510196,2131-01-12 17:40:00,227194,65.0,Planned Extubation,2131-01-11 04:30:00,224385.0,65.0,1 days 13:10:00
5,10003400,23559586,38383343,2137-08-21 15:40:00,227194,99.6,Planned Extubation,2137-08-17 21:21:00,224385.0,90.5,3 days 18:19:00
7,10004401,29988601,32773003,2144-01-30 12:30:00,227194,76.0,Planned Extubation,2144-01-27 19:00:00,224385.0,76.0,2 days 17:30:00
15,10005817,28661809,31316840,2135-01-19 14:00:00,227194,88.8,Planned Extubation,2135-01-06 08:33:00,224385.0,88.8,13 days 05:27:00
30,10015931,22130791,37093652,2177-03-29 13:11:00,227194,59.5,Planned Extubation,2177-03-25 09:57:00,224385.0,59.5,4 days 03:14:00
...,...,...,...,...,...,...,...,...,...,...,...
25002,19997367,20617667,35616526,2126-05-02 16:30:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,5 days 20:16:00
25004,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,11 days 21:15:00
25005,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-05-05 18:20:00,224385.0,59.0,2 days 23:09:00
25008,19999068,21606769,30143796,2161-08-28 13:35:00,227194,55.8,Planned Extubation,2161-08-25 15:34:00,224385.0,55.8,2 days 22:01:00


In [173]:
#위 표에서 extubationintubationtime은 정수 시간으로 계산되어야 하고, 이를 위해 timedelta를 사용할 것임, 
#그리고 이를 시간으로 바꿔주기 위해 total_seconds를 사용할 것임
extubation_intubation2['extubationtointubationtime1'] = extubation_intubation2['extubationtointubationtime'].dt.total_seconds()/3600

In [174]:
extubation_intubation2.rename(columns={'extubationtointubationtime1':'ventilator_duration'}, inplace=True)

In [175]:
extubation_intubation2 #6566 개의 row만 남음

,subject_id,hadm_id,stay_id,extubationtime,int_itemid,int_weight,extubationcause,intubationtime,ext_itemid,ext_weight,extubationtointubationtime,ventilator_duration
0,10001884,26184834,37510196,2131-01-12 17:40:00,227194,65.0,Planned Extubation,2131-01-11 04:30:00,224385.0,65.0,1 days 13:10:00,37.166667
5,10003400,23559586,38383343,2137-08-21 15:40:00,227194,99.6,Planned Extubation,2137-08-17 21:21:00,224385.0,90.5,3 days 18:19:00,90.316667
7,10004401,29988601,32773003,2144-01-30 12:30:00,227194,76.0,Planned Extubation,2144-01-27 19:00:00,224385.0,76.0,2 days 17:30:00,65.500000
15,10005817,28661809,31316840,2135-01-19 14:00:00,227194,88.8,Planned Extubation,2135-01-06 08:33:00,224385.0,88.8,13 days 05:27:00,317.450000
30,10015931,22130791,37093652,2177-03-29 13:11:00,227194,59.5,Planned Extubation,2177-03-25 09:57:00,224385.0,59.5,4 days 03:14:00,99.233333
...,...,...,...,...,...,...,...,...,...,...,...,...
25002,19997367,20617667,35616526,2126-05-02 16:30:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,5 days 20:16:00,140.266667
25004,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,11 days 21:15:00,285.250000
25005,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-05-05 18:20:00,224385.0,59.0,2 days 23:09:00,71.150000
25008,19999068,21606769,30143796,2161-08-28 13:35:00,227194,55.8,Planned Extubation,2161-08-25 15:34:00,224385.0,55.8,2 days 22:01:00,70.016667


In [176]:
#여기서 최소 24시간 이상 ventilator 를 유지한 케이스만 남길 것임
extubation_intubation3 = extubation_intubation2 >> mask(extubation_intubation2.ventilator_duration >= 24)

In [177]:
extubation_intubation3 #4568 case만 남음

,subject_id,hadm_id,stay_id,extubationtime,int_itemid,int_weight,extubationcause,intubationtime,ext_itemid,ext_weight,extubationtointubationtime,ventilator_duration
0,10001884,26184834,37510196,2131-01-12 17:40:00,227194,65.0,Planned Extubation,2131-01-11 04:30:00,224385.0,65.0,1 days 13:10:00,37.166667
5,10003400,23559586,38383343,2137-08-21 15:40:00,227194,99.6,Planned Extubation,2137-08-17 21:21:00,224385.0,90.5,3 days 18:19:00,90.316667
7,10004401,29988601,32773003,2144-01-30 12:30:00,227194,76.0,Planned Extubation,2144-01-27 19:00:00,224385.0,76.0,2 days 17:30:00,65.500000
15,10005817,28661809,31316840,2135-01-19 14:00:00,227194,88.8,Planned Extubation,2135-01-06 08:33:00,224385.0,88.8,13 days 05:27:00,317.450000
30,10015931,22130791,37093652,2177-03-29 13:11:00,227194,59.5,Planned Extubation,2177-03-25 09:57:00,224385.0,59.5,4 days 03:14:00,99.233333
...,...,...,...,...,...,...,...,...,...,...,...,...
24997,19989918,20256395,35582926,2175-09-26 16:00:00,227194,124.4,Planned Extubation,2175-09-23 14:15:00,224385.0,124.4,3 days 01:45:00,73.750000
25002,19997367,20617667,35616526,2126-05-02 16:30:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,5 days 20:16:00,140.266667
25004,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-04-26 20:14:00,224385.0,59.0,11 days 21:15:00,285.250000
25005,19997367,20617667,35616526,2126-05-08 17:29:00,227194,59.0,Planned Extubation,2126-05-05 18:20:00,224385.0,59.0,2 days 23:09:00,71.150000


In [178]:
#tracheostomy 도 intubation 데이터에 넣어야 할까??? ---> 아님, tracheostomy 는 내가 보고자 하는 문제에 속하지 않는 케이스임

In [179]:
ventilator_duration1 = extubation_intubation3 >> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','ventilator_duration','int_itemid','ext_itemid','int_weight','ext_weight')

In [180]:
ventilator_duration1

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0
5,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5
7,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0
15,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8
30,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,99.233333,227194,224385.0,59.5,59.5
...,...,...,...,...,...,...,...,...,...,...,...
24997,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,73.750000,227194,224385.0,124.4,124.4
25002,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,140.266667,227194,224385.0,59.0,59.0
25004,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0
25005,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0


In [181]:
ventilator_duration1.to_csv('../outputs/ventilator_duration1.csv', index=False)
#ventilator_duration1 = pd.read_csv('../outputs/ventilator_duration1.csv')
#ventilator_duration1

In [182]:
#이제 여기에 intubation1을 다시 붙여서 reintubation time 을 만들어야 함
ventilator_duration2 = ventilator_duration1 >> left_join(intubation1, by = ("subject_id", "hadm_id", "stay_id"))

In [183]:
ventilator_duration2

,subject_id,hadm_id,stay_id,intubationtime_x,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,intubationtime_y,itemid,patientweight
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0,2131-01-11 04:30:00,224385,65.0
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5,2137-08-17 21:21:00,224385,90.5
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-27 19:00:00,224385,76.0
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0
4,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8,2135-01-06 08:33:00,224385,88.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0
6996,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0
6997,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0
6998,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0


In [184]:
ventilator_duration2.dtypes

subject_id                      int64
hadm_id                         int64
stay_id                         int64
intubationtime_x       datetime64[ns]
extubationtime         datetime64[ns]
extubationcause                object
ventilator_duration           float64
int_itemid                      int64
ext_itemid                    float64
int_weight                    float64
ext_weight                    float64
intubationtime_y               object
itemid                          int64
patientweight                 float64
dtype: object

In [185]:
ventilator_duration2.rename(columns={'intubationtime_x':'intubationtime', 'intubationtime_y':'reintubationtime', 'itemid':'rein_itemid','patientweight':'rein_weight'}, inplace=True)

In [186]:
ventilator_duration2

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,reintubationtime,rein_itemid,rein_weight
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0,2131-01-11 04:30:00,224385,65.0
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5,2137-08-17 21:21:00,224385,90.5
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-27 19:00:00,224385,76.0
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0
4,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8,2135-01-06 08:33:00,224385,88.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0
6996,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0
6997,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0
6998,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0


In [193]:
ventilator_duration2['reintubationtime'] = pd.to_datetime(ventilator_duration2['reintubationtime'])

In [194]:
ventilator_duration2['reintubation_duration'] = ventilator_duration2['reintubationtime'] - ventilator_duration2['extubationtime']

In [195]:
ventilator_duration2 #이것도 reintubation_duration이 음수인 것은 reintubation time이 extubation time보다 작은 경우이므로 지워줘야 함

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,reintubationtime,rein_itemid,rein_weight,reintubation_duration
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0,2131-01-11 04:30:00,224385,65.0,-2 days +10:50:00
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5,2137-08-17 21:21:00,224385,90.5,-4 days +05:41:00
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-27 19:00:00,224385,76.0,-3 days +06:30:00
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0,0 days 00:30:00
4,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8,2135-01-06 08:33:00,224385,88.8,-14 days +18:33:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0,-12 days +02:45:00
6996,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0,-3 days +00:51:00
6997,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-04-26 20:14:00,224385,59.0,-12 days +02:45:00
6998,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0,2126-05-05 18:20:00,224385,59.0,-3 days +00:51:00


In [196]:
ventilator_duration2.dtypes

subject_id                         int64
hadm_id                            int64
stay_id                            int64
intubationtime            datetime64[ns]
extubationtime            datetime64[ns]
extubationcause                   object
ventilator_duration              float64
int_itemid                         int64
ext_itemid                       float64
int_weight                       float64
ext_weight                       float64
reintubationtime          datetime64[ns]
rein_itemid                        int64
rein_weight                      float64
reintubation_duration    timedelta64[ns]
dtype: object

In [197]:
ventilator_duration3 = ventilator_duration2 >> mask(ventilator_duration2.reintubation_duration > timedelta(days=0))

In [198]:
ventilator_duration3

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,reintubationtime,rein_itemid,rein_weight,reintubation_duration
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0,0 days 00:30:00
14,10032381,20176432,34622731,2115-07-06 02:15:00,2115-07-15 11:00:00,Planned Extubation,224.750000,227194,224385.0,78.0,78.0,2115-07-18 17:00:00,224385,78.0,3 days 06:00:00
16,10039708,28258130,33281088,2140-02-01 16:30:00,2140-02-02 17:40:00,Planned Extubation,25.166667,227194,224385.0,49.8,49.8,2140-02-02 19:03:00,224385,49.8,0 days 01:23:00
33,10098215,22394571,32301420,2118-06-18 01:48:00,2118-06-21 03:30:00,Unplanned Extuabtion (patient-initiated),73.700000,225468,224385.0,48.7,48.7,2118-06-21 08:28:00,224385,48.7,0 days 04:58:00
39,10098215,22394571,36018186,2118-06-03 20:50:00,2118-06-06 16:33:00,Planned Extubation,67.716667,227194,224385.0,55.4,55.4,2118-06-07 01:40:00,224385,55.4,0 days 09:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-17 09:00:00,Planned Extubation,124.750000,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,10 days 10:51:00
6963,19913577,20355379,37544707,2114-05-21 07:53:00,2114-05-27 13:10:00,Planned Extubation,149.283333,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00
6966,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-27 13:10:00,Planned Extubation,368.916667,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00
6968,19928728,21394753,32643307,2177-09-23 07:16:00,2177-09-25 15:17:00,Planned Extubation,56.016667,227194,224385.0,53.5,53.5,2177-09-26 20:15:00,224385,53.5,1 days 04:58:00


In [199]:
ventilator_duration3['reintubation_duration1'] = ventilator_duration3['reintubation_duration'].dt.total_seconds()/3600

In [200]:
ventilator_duration3

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,reintubationtime,rein_itemid,rein_weight,reintubation_duration,reintubation_duration1
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0,0 days 00:30:00,0.500000
14,10032381,20176432,34622731,2115-07-06 02:15:00,2115-07-15 11:00:00,Planned Extubation,224.750000,227194,224385.0,78.0,78.0,2115-07-18 17:00:00,224385,78.0,3 days 06:00:00,78.000000
16,10039708,28258130,33281088,2140-02-01 16:30:00,2140-02-02 17:40:00,Planned Extubation,25.166667,227194,224385.0,49.8,49.8,2140-02-02 19:03:00,224385,49.8,0 days 01:23:00,1.383333
33,10098215,22394571,32301420,2118-06-18 01:48:00,2118-06-21 03:30:00,Unplanned Extuabtion (patient-initiated),73.700000,225468,224385.0,48.7,48.7,2118-06-21 08:28:00,224385,48.7,0 days 04:58:00,4.966667
39,10098215,22394571,36018186,2118-06-03 20:50:00,2118-06-06 16:33:00,Planned Extubation,67.716667,227194,224385.0,55.4,55.4,2118-06-07 01:40:00,224385,55.4,0 days 09:07:00,9.116667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-17 09:00:00,Planned Extubation,124.750000,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,10 days 10:51:00,250.850000
6963,19913577,20355379,37544707,2114-05-21 07:53:00,2114-05-27 13:10:00,Planned Extubation,149.283333,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00,6.683333
6966,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-27 13:10:00,Planned Extubation,368.916667,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00,6.683333
6968,19928728,21394753,32643307,2177-09-23 07:16:00,2177-09-25 15:17:00,Planned Extubation,56.016667,227194,224385.0,53.5,53.5,2177-09-26 20:15:00,224385,53.5,1 days 04:58:00,28.966667


In [201]:
ventilator_duration3.to_csv('../outputs/ventilator_duration3.csv', index=False)
#ventilator_duration3 = pd.read_csv('../outputs/ventilator_duration3.csv')
#ventilator_duration3

In [ ]:
#ventilation_duration1의 4568 건 중, (intubation-extubation)
#891건은 intubation-extubation-reintubation 이 있는 이벤트임
#그럼 방법은 크게 2가지로 할 수 있는데, 
#1. extubation이 있는 환자의 reintubation 이벤트가 있는 환자들 데이터(891)건만 사용해서 이 안에서 extubation의 성공/실패 여부를 보는 것과 [기존방법] -- subject_subset1
#2. extubation 이후 reintubation 이 없는 환자들을 extubation non-failure로 넣어서 함께 분류하는 방법이 있음. 이 경우, 사망이나 퇴원등의 이벤트가 조회되어야 사용할 수 있을 것임.
#1.에서는 한사람이 여러번의 케이스를 수행할 수 있으므로 그 상관관계도 확인한다면 더 의미있을지도 모름(1-2)

In [202]:
ventilator_duration1

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0
5,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5
7,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0
15,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8
30,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,99.233333,227194,224385.0,59.5,59.5
...,...,...,...,...,...,...,...,...,...,...,...
24997,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,73.750000,227194,224385.0,124.4,124.4
25002,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,140.266667,227194,224385.0,59.0,59.0
25004,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0
25005,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0


In [203]:
ventilator_duration1.value_counts('subject_id')

subject_id
19321668    13
10098215    13
13276959     9
16402020     9
16702834     8
            ..
13786618     1
13786794     1
13787816     1
13792260     1
19999068     1
Name: count, Length: 3282, dtype: int64

In [204]:
ventilator_duration3

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight,reintubationtime,rein_itemid,rein_weight,reintubation_duration,reintubation_duration1
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0,2144-01-30 13:00:00,224385,76.0,0 days 00:30:00,0.500000
14,10032381,20176432,34622731,2115-07-06 02:15:00,2115-07-15 11:00:00,Planned Extubation,224.750000,227194,224385.0,78.0,78.0,2115-07-18 17:00:00,224385,78.0,3 days 06:00:00,78.000000
16,10039708,28258130,33281088,2140-02-01 16:30:00,2140-02-02 17:40:00,Planned Extubation,25.166667,227194,224385.0,49.8,49.8,2140-02-02 19:03:00,224385,49.8,0 days 01:23:00,1.383333
33,10098215,22394571,32301420,2118-06-18 01:48:00,2118-06-21 03:30:00,Unplanned Extuabtion (patient-initiated),73.700000,225468,224385.0,48.7,48.7,2118-06-21 08:28:00,224385,48.7,0 days 04:58:00,4.966667
39,10098215,22394571,36018186,2118-06-03 20:50:00,2118-06-06 16:33:00,Planned Extubation,67.716667,227194,224385.0,55.4,55.4,2118-06-07 01:40:00,224385,55.4,0 days 09:07:00,9.116667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-17 09:00:00,Planned Extubation,124.750000,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,10 days 10:51:00,250.850000
6963,19913577,20355379,37544707,2114-05-21 07:53:00,2114-05-27 13:10:00,Planned Extubation,149.283333,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00,6.683333
6966,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-27 13:10:00,Planned Extubation,368.916667,227194,224385.0,85.0,85.0,2114-05-27 19:51:00,224385,85.0,0 days 06:41:00,6.683333
6968,19928728,21394753,32643307,2177-09-23 07:16:00,2177-09-25 15:17:00,Planned Extubation,56.016667,227194,224385.0,53.5,53.5,2177-09-26 20:15:00,224385,53.5,1 days 04:58:00,28.966667


In [205]:
ventilator_duration3.value_counts('subject_id')

subject_id
13033327    13
19321668    13
16337817    10
13462065     9
15227574     7
            ..
13821485     1
13808136     1
13804231     1
13794644     1
19997367     1
Name: count, Length: 521, dtype: int64

In [206]:
ventilator_duration3.value_counts('extubationcause')

extubationcause
Planned Extubation                              748
Unplanned Extuabtion (patient-initiated)        127
Unplanned Extuabtion (non-patient initiated)     16
Name: count, dtype: int64

In [207]:
ventilator_duration4 = ventilator_duration3 >> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','ventilator_duration','reintubationtime','reintubation_duration1','int_itemid','ext_itemid','int_weight','ext_weight','rein_weight')

In [208]:
ventilator_duration4.rename(columns={'reintubation_duration1':'reintubation_duration'}, inplace=True)

In [209]:
ventilator_duration4['extubationfailure']=ventilator_duration4['reintubation_duration'].apply(lambda x: 1 if x <= 48 else 0)

In [210]:
ventilator_duration4

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,reintubationtime,reintubation_duration,int_itemid,ext_itemid,int_weight,ext_weight,rein_weight,extubationfailure
3,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,2144-01-30 13:00:00,0.500000,227194,224385.0,76.0,76.0,76.0,1
14,10032381,20176432,34622731,2115-07-06 02:15:00,2115-07-15 11:00:00,Planned Extubation,224.750000,2115-07-18 17:00:00,78.000000,227194,224385.0,78.0,78.0,78.0,0
16,10039708,28258130,33281088,2140-02-01 16:30:00,2140-02-02 17:40:00,Planned Extubation,25.166667,2140-02-02 19:03:00,1.383333,227194,224385.0,49.8,49.8,49.8,1
33,10098215,22394571,32301420,2118-06-18 01:48:00,2118-06-21 03:30:00,Unplanned Extuabtion (patient-initiated),73.700000,2118-06-21 08:28:00,4.966667,225468,224385.0,48.7,48.7,48.7,1
39,10098215,22394571,36018186,2118-06-03 20:50:00,2118-06-06 16:33:00,Planned Extubation,67.716667,2118-06-07 01:40:00,9.116667,227194,224385.0,55.4,55.4,55.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-17 09:00:00,Planned Extubation,124.750000,2114-05-27 19:51:00,250.850000,227194,224385.0,85.0,85.0,85.0,0
6963,19913577,20355379,37544707,2114-05-21 07:53:00,2114-05-27 13:10:00,Planned Extubation,149.283333,2114-05-27 19:51:00,6.683333,227194,224385.0,85.0,85.0,85.0,1
6966,19913577,20355379,37544707,2114-05-12 04:15:00,2114-05-27 13:10:00,Planned Extubation,368.916667,2114-05-27 19:51:00,6.683333,227194,224385.0,85.0,85.0,85.0,1
6968,19928728,21394753,32643307,2177-09-23 07:16:00,2177-09-25 15:17:00,Planned Extubation,56.016667,2177-09-26 20:15:00,28.966667,227194,224385.0,53.5,53.5,53.5,1


In [211]:
ventilator_duration4.value_counts('extubationfailure')

extubationfailure
1    582
0    309
Name: count, dtype: int64

In [212]:
#subject_subset1 은 reintubation event가 있는 사람들 중에서, extubation failure 여부를 표시한 것임 total n수는 891 이며, extubation failure는 582건, extubation non-failure는 309건임.
#또한, extubationcause는
#Planned Extubation                              748
#Unplanned Extuabtion (patient-initiated)        127
#Unplanned Extuabtion (non-patient initiated)     16
ventilator_duration4.to_csv('../outputs/subject_subset1.csv', index=False)

In [213]:
ventilator_duration1
#여기서 outcome이라는 컬럼을 만들어서, reintubation, discharged, death, unknown 을 넣어 줄 것임.

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,ventilator_duration,int_itemid,ext_itemid,int_weight,ext_weight
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,37.166667,227194,224385.0,65.0,65.0
5,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,90.316667,227194,224385.0,99.6,90.5
7,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,65.500000,227194,224385.0,76.0,76.0
15,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,317.450000,227194,224385.0,88.8,88.8
30,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,99.233333,227194,224385.0,59.5,59.5
...,...,...,...,...,...,...,...,...,...,...,...
24997,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,73.750000,227194,224385.0,124.4,124.4
25002,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,140.266667,227194,224385.0,59.0,59.0
25004,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,285.250000,227194,224385.0,59.0,59.0
25005,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,71.150000,227194,224385.0,59.0,59.0


In [214]:
#admission -- dischtime, deathtime 있음 transfer 는 병동으로 간거고, icustay는 icu 나간거임.

In [215]:
admissions.head(2)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0


In [216]:
icustays.head(2)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535


In [217]:
ventilator_duration1_1 = ventilator_duration1 >> left_join(admissions, by = ("subject_id", "hadm_id")) \
>> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','ventilator_duration','int_weight','ext_weight','dischtime','deathtime') \
>> left_join(icustays, by = ("subject_id", "hadm_id","stay_id"))

In [218]:
ventilator_duration1_2 = ventilator_duration1_1 >> left_join(ventilator_duration4, by=('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','ventilator_duration','int_weight','ext_weight')) \
>> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','reintubationtime','reintubation_duration','extubationfailure','ventilator_duration','int_weight','ext_weight','dischtime','deathtime','first_careunit','last_careunit','intime','outtime','los')

In [219]:
#outcome을 정리해야함. reintubation, discharged, death, unknown
#reintubation -- reintubationtime이 있으면 reintubation
#reintubation으로 분류되지 않은 것 중에, dischtime이 있으면 discharged, deathtime이 있으면 death, 둘다 없으면 unknown
#일단 이 조건으로 걸어보자

def label_outcomes(row):
    if  pd.notnull(row['reintubationtime']):
        return 'reintubation'
    elif pd.notnull(row['deathtime']):
        return 'death'
    elif pd.notnull(row['dischtime']):
        return 'discharged'
    else:
        return 'unknown'

In [220]:
ventilator_duration1_2['outcome_step1'] = ventilator_duration1_2.apply(lambda row: label_outcomes(row), axis=1)

In [221]:
ventilator_duration1_2

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,extubationfailure,ventilator_duration,int_weight,ext_weight,dischtime,deathtime,first_careunit,last_careunit,intime,outtime,los,outcome_step1
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,NaN,37.166667,65.0,65.0,2131-01-20 05:15:00,2131-01-20 05:15:00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,death
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,NaT,NaN,NaN,90.316667,99.6,90.5,2137-09-02 17:05:00,2137-09-02 17:05:00,Coronary Care Unit (CCU),Medical Intensive Care Unit (MICU),2137-08-17 17:36:37,2137-09-02 19:17:11,16.069838,death
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,2144-01-30 13:00:00,0.500000,1.0,65.500000,76.0,76.0,2144-02-06 11:45:00,NaT,Medical Intensive Care Unit (MICU),Trauma SICU (TSICU),2144-01-26 22:28:04,2144-02-06 13:44:15,10.636238,reintubation
3,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,NaT,NaN,NaN,317.450000,88.8,88.8,2135-01-19 18:36:00,2135-01-19 18:36:00,Medical Intensive Care Unit (MICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2135-01-03 21:55:32,2135-01-19 21:16:23,15.972812,death
4,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,NaT,NaN,NaN,99.233333,59.5,59.5,2177-03-29 14:15:00,2177-03-29 14:15:00,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2177-03-24 21:48:07,2177-03-29 18:03:36,4.844086,death
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,NaT,NaN,NaN,73.750000,124.4,124.4,2175-09-28 17:45:00,NaT,Trauma SICU (TSICU),Trauma SICU (TSICU),2175-09-22 18:17:41,2175-09-27 11:53:51,4.733449,discharged
4728,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,2126-05-05 18:20:00,73.833333,0.0,140.266667,59.0,59.0,2126-05-19 14:15:00,NaT,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2126-04-21 10:09:25,2126-05-16 18:15:31,25.337569,reintubation
4729,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,NaN,285.250000,59.0,59.0,2126-05-19 14:15:00,NaT,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2126-04-21 10:09:25,2126-05-16 18:15:31,25.337569,discharged
4730,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,NaN,71.150000,59.0,59.0,2126-05-19 14:15:00,NaT,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2126-04-21 10:09:25,2126-05-16 18:15:31,25.337569,discharged


In [222]:
ventilator_duration1_3 = ventilator_duration1_2 >> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','extubationcause','reintubationtime','reintubation_duration','dischtime','deathtime','outcome_step1') 

In [223]:
ventilator_duration1_3.value_counts('outcome_step1') #이게아님

outcome_step1
discharged      2887
death            936
reintubation     909
Name: count, dtype: int64

In [224]:
ventilator_duration1_3

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,dischtime,deathtime,outcome_step1
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,NaT,NaN,2137-09-02 17:05:00,2137-09-02 17:05:00,death
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,2144-01-30 13:00:00,0.500000,2144-02-06 11:45:00,NaT,reintubation
3,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,NaT,NaN,2135-01-19 18:36:00,2135-01-19 18:36:00,death
4,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,NaT,NaN,2177-03-29 14:15:00,2177-03-29 14:15:00,death
...,...,...,...,...,...,...,...,...,...,...,...
4727,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,NaT,NaN,2175-09-28 17:45:00,NaT,discharged
4728,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,2126-05-05 18:20:00,73.833333,2126-05-19 14:15:00,NaT,reintubation
4729,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged
4730,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged


In [225]:
#위에서 outcome_step1이 death일 때, death와 extubationtime 의 날짜가 같거나(시간은 같지 않아도 됨)은 경우, outcome_step2에서  death로 분류하고
#그렇지 않은 경우에는 unknown으로 분류할 것임
#위에서 discharged로 분류된 경우에는 dischtime과 extubationtime이 같거나(시간은 같지 않아도 됨)은 경우, outcome_step2에서 discharged로 분류하고
#그렇지 않은 경우에는 unknown으로 분류할 것임


def classify_outcome_step2_date(row):
    if row['outcome_step1'] == 'reintubation':
        return 'reintubation'
    elif row['outcome_step1'] == 'death' and row['deathtime'].date() == row['extubationtime'].date():
        return 'death'
    elif row['outcome_step1'] == 'discharged' and row['dischtime'].date() == row['extubationtime'].date():
        return 'discharged'
    else:
        return 'unknown'

In [226]:
ventilator_duration1_3['outcome_step2'] = ventilator_duration1_3.apply(lambda row: classify_outcome_step2_date(row), axis=1)

In [227]:
ventilator_duration1_3.value_counts('outcome_step2')

outcome_step2
unknown         3394
reintubation     909
death            412
discharged        17
Name: count, dtype: int64

In [228]:
#위에서 outcome_step1이 death일 때, deathtime 24시간 이내, extubationtime 이 있으면, outcome_step2에서  death로 분류하고
#그렇지 않은 경우에는 unknown으로 분류할 것임
#위에서 discharged로 분류된 경우에는 dischtime과 24시간 이내 extubationtime이 있으면, outcome_step2에서 discharged로 분류하고
#그렇지 않은 경우에는 unknown으로 분류할 것임

# from datetime import timedelta

# def classify_outcome_step2_24(row):
#     if row['outcome_step1'] == 'reintubation':
#         return 'reintubation'
#     elif row['outcome_step1'] == 'death' and abs(row['deathtime'] - row['extubationtime']) <= timedelta(hours=24):
#         return 'death'
#     elif row['outcome_step1'] == 'discharged' and abs(row['dischtime'] - row['extubationtime']) <= timedelta(hours=24):
#         return 'discharged'
#     else:
#         return 'unknown'



In [229]:
# ventilator_duration1_3['outcome_step3'] = ventilator_duration1_3.apply(lambda row: classify_outcome_step2_24(row), axis=1)


In [230]:
# ventilator_duration1_3.value_counts('outcome_step3')

In [231]:
#24시간의 delay 를 주기보다 같은 날짜일때만 하기로 했음
#그렇다면 어떤것을 extubation failure로 분류할 것인가? 라는 문제가 생기는데,
#reintubation의 case 에서는, extubation failure로 분류되는 것은 reintubation이 48시간 이내에 일어난 경우임, reintubation time 이 48시간을 초과하면 non-failure임
#death 와 discharge는 제외하고 생각해야 할까? 아니면 failure에 포함해야 할까? 
#death는 만약에 discharge와,unknown만 쓸거라면 사망 24시간이내는 버리는게 맞을지도(사망 24시간 동안은 condition이 나쁠 것이므로,)
#discharge는 충분히 환기가 가능하다고 생각하여 뽑고 퇴원했을 것이라고 가정하면 non-failure가 될 수 있을 듯.
#unknown 은 non-failure 로 할 것인지? 근데 extubation event가 있고 나서, reintubation event자체가 발생하지 않았다면 non-failure로 분류하는게 맞을 듯
#이건 상의해보고 결정
#unplanned extubation은 빼야할지? 이것도 고려 필요

In [232]:
#reintubation => reintubation
#extubation 후 24시간 이내 사망한 대상자는 포함하지 않음. death => death 
#Death는 두 경우로 나누어 볼 수 있음 ------ 48시간 이내 사망은 failure로 하고(그럼48시간 이내 재삽관 또는 사망은 failure), on date 사망은 (사망했기 때문에 관을 제거한것이므로 선후 관계가 잘못됨.아예제거해야 할 데이터)
#death가 아닌 discharge는 non-failure로 분류. discharge => non-failure
#unknown은 non-faiure 로 분류. unknown => non-failure (extubation 후 이후 이벤트 없음)

def classify_outcome_step2(row):
    if row['outcome_step1'] == 'reintubation' and (row['reintubation_duration']) <= 48:
        return 'reintubation (extubation failure)'
    elif row['outcome_step1'] == 'reintubation' and (row['reintubation_duration']) > 48:
        return 'reintubation (extubation non-failure)'
    elif row['outcome_step1'] == 'death' and (row['deathtime'] - row['extubationtime']) <= timedelta(hours=0):
        return 'ext after death'
    elif row['outcome_step1'] == 'death' and (row['deathtime'] - row['extubationtime']) <= timedelta(hours=48): 
        return 'within 48hour death'
    elif row['outcome_step1'] == 'discharged' and row['dischtime'].date() == row['extubationtime'].date():
        return 'discharged'
    else:
        return 'unknown'

In [233]:
ventilator_duration1_3['outcome_step3'] = ventilator_duration1_3.apply(lambda row: classify_outcome_step2(row), axis=1)

In [234]:
ventilator_duration1_3 >> filter_by(X.outcome_step3 == 'ext after death')

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,dischtime,deathtime,outcome_step1,outcome_step2,outcome_step3
133,10280129,21648590,37686710,2171-03-25 02:52:00,2171-03-27 14:10:00,Planned Extubation,NaT,NaN,2171-03-27 14:07:00,2171-03-27 14:07:00,death,death,ext after death
552,11092156,25081268,35199653,2182-07-16 11:00:00,2182-07-27 17:20:00,Planned Extubation,NaT,NaN,2182-07-26 12:30:00,2182-07-26 12:31:00,death,unknown,ext after death
574,11134447,29068519,30423942,2115-12-04 16:00:00,2115-12-12 11:50:00,Planned Extubation,NaT,NaN,2115-12-12 00:01:00,2115-12-12 00:01:00,death,death,ext after death
683,11401579,29426063,37043634,2127-09-10 03:17:00,2127-09-12 01:00:00,Planned Extubation,NaT,NaN,2127-09-10 01:30:00,2127-09-10 13:42:00,death,unknown,ext after death
717,11438221,25159418,33121620,2169-11-21 02:09:00,2169-11-26 03:11:00,Planned Extubation,NaT,NaN,2169-11-26 03:08:00,2169-11-26 03:08:00,death,death,ext after death
731,11452644,29901909,35689867,2129-11-15 10:48:00,2129-11-19 21:51:00,Planned Extubation,NaT,NaN,2129-11-19 21:51:00,2129-11-19 21:51:00,death,death,ext after death
951,11959638,24816314,37064466,2178-09-13 10:00:00,2178-09-27 09:15:00,Planned Extubation,NaT,NaN,2178-09-27 09:15:00,2178-09-27 09:15:00,death,death,ext after death
1041,12116228,20421390,30958024,2122-02-02 16:00:00,2122-02-05 04:00:00,Planned Extubation,NaT,NaN,2122-02-05 04:00:00,2122-02-05 04:00:00,death,death,ext after death
1270,12731441,25429738,36033368,2141-10-09 16:17:00,2141-10-11 14:30:00,Planned Extubation,NaT,NaN,2141-10-11 14:08:00,2141-10-11 14:08:00,death,death,ext after death
1628,13411545,20440298,36125394,2176-04-09 09:00:00,2176-04-24 15:00:00,Planned Extubation,NaT,NaN,2176-04-24 14:15:00,2176-04-24 14:15:00,death,death,ext after death


In [240]:
ventilator_duration1_3.to_csv('../outputs/subject_subset2.csv', index=False)

In [235]:
ventilator_duration1_3.value_counts('outcome_step3') #within48 hr death 로 하니까 discharge 에서 death 로 많이 옮겨왔네, 
# unknown                3177 (non-failure) ==> 뒤에 이벤트 없이 쭉 있다가 퇴원했다면, 퇴원날짜로 end time 하던지 하면 되나...?
# reintubation(failure)    594 (failure)
# reintubation(non-failure)  315 (non-failure)
# within 48hour death     584 (failure)
# ext after death          45 ---> eliminate
# discharged               17 (non-failure)

outcome_step3
unknown                                  3177
reintubation (extubation failure)         594
within 48hour death                       584
reintubation (extubation non-failure)     315
ext after death                            45
discharged                                 17
Name: count, dtype: int64

In [236]:
ventilator_duration1_3 >> filter_by(X.outcome_step3 == 'unknown')

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,dischtime,deathtime,outcome_step1,outcome_step2,outcome_step3
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,NaT,NaN,2137-09-02 17:05:00,2137-09-02 17:05:00,death,unknown,unknown
5,10020740,23831430,31077365,2150-03-30 08:20:00,2150-04-02 01:09:00,Planned Extubation,NaT,NaN,2150-04-25 13:50:00,NaT,discharged,unknown,unknown
8,10024982,25154057,37919901,2203-10-02 11:00:00,2203-10-09 14:50:00,Planned Extubation,NaT,NaN,2203-10-11 19:10:00,2203-10-11 19:10:00,death,unknown,unknown
9,10027602,28166872,32391858,2201-11-07 15:00:00,2201-11-09 12:45:00,Planned Extubation,NaT,NaN,2201-11-20 14:45:00,NaT,discharged,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4726,19984259,20109446,37233855,2153-12-01 11:49:00,2153-12-05 13:10:00,Planned Extubation,NaT,NaN,2153-12-08 17:05:00,NaT,discharged,unknown,unknown
4727,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,NaT,NaN,2175-09-28 17:45:00,NaT,discharged,unknown,unknown
4729,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged,unknown,unknown
4730,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged,unknown,unknown


In [237]:
tunnel.close() #터널 더이상 사용하지 않을때 호출해줘야 합니다.